# Lex

In this notebook, we will learn how to create a deterministic bot using Amazon Lex, the AWS service for conversational bots based on predefined rules and flows.

A deterministic bot follows structured conversation flows, defined by intents, slots, and utterances, unlike generative models that produce open-ended responses. This makes it ideal for cases where responses must be precise and predictable.

Notebook contents

- Initialising the AWS Lex client using boto3.

- Creating a bot and configuring intents.

- Defining utterances and slots.

- Testing interaction with the bot

In [9]:
import boto3
import json
import time

lexv2 = boto3.client('lexv2-models', region_name='eu-west-1')
lex_runtime = boto3.client('lexv2-runtime', region_name='eu-west-1')

In [10]:
# utilizamos el mismo role creado anteriormente para agentes, ya que le dimos full_access
role_arn = "arn:aws:iam::XXXXXXXXX:role/AmazonBedrockExecutionRoleForAgents_notifier-agent"

# Define Bot

In [11]:
bot_name = "DemoBot"

In [ ]:
response = lexv2.create_bot(
    botName=bot_name,
    description="Bot creado desde un notebook de SageMaker",
    roleArn=role_arn, 
    dataPrivacy={'childDirected': False},
    idleSessionTTLInSeconds=300
)

bot_id = response['botId']
print("Bot creado con ID:", bot_id)

# Personalizar Bot

In [13]:
response = lexv2.create_bot_locale(
    botId=bot_id,
    botVersion="DRAFT",
    localeId="es_ES",
    nluIntentConfidenceThreshold=0.4,
    voiceSettings={'voiceId': 'Lucia'}  # voz opcional
)

locale_id = response['localeId']
locale_id

'es_ES'

# Intent

Lex bots can identify what the user wants through *intent*.

In [ ]:
intent_name = "ReservarMesaIntent"

intent_response = lexv2.create_intent(
    botId=bot_id,
    botVersion="DRAFT",
    localeId=locale_id,
    intentName=intent_name,
    sampleUtterances=[
      {"utterance":"Quiero reservar una mesa"},
      {"utterance":"Hacer una reserva"}
    ],
    intentClosingSetting={
        "closingResponse":{
            "messageGroups":[{
                "message":{"plainTextMessage":{"value":"¡Reserva confirmada!"}}
            }]
        }
    }
)

intent_id = intent_response['intentId']
intent_id

# Build Bot

In [ ]:
build = lexv2.build_bot_locale(
    botId=bot_id,
    botVersion="DRAFT",
    localeId=locale_id
)

build

In [ ]:
check = lexv2.describe_bot_locale(
        botId=bot_id, botVersion="DRAFT", localeId=locale_id
    )
check['botLocaleStatus']

# Test

In [ ]:
response = lex_runtime.recognize_text(
    botId=bot_id,
    botAliasId='TSTALIASID',  # funciona con cualquiera
    localeId='es_ES',
    sessionId='test123',
    text='Quiero reservar una mesa'
)

response

# Añadimos más intents

In [22]:
intent_cancel = lexv2.create_intent(
    botId=bot_id,
    botVersion="DRAFT",
    localeId=locale_id,
    intentName="CancelarReservaIntent",
    sampleUtterances=[
        {"utterance": "Quiero cancelar mi reserva"},
        {"utterance": "Cancelar reserva"}
    ],
    intentClosingSetting={
        "closingResponse":{
            "messageGroups":[{
                "message":{"plainTextMessage":{"value":"Tu reserva ha sido cancelada."}}
            }]
        }
    }
)
intent_cancel_id = intent_cancel['intentId']

In [ ]:
intent_cancel_id

In [ ]:
# actualizamos bot
lexv2.build_bot_locale(botId=bot_id, botVersion="DRAFT", localeId=locale_id)


In [ ]:
response = lex_runtime.recognize_text(
    botId=bot_id,
    botAliasId='TSTALIASID',  # funciona con cualquiera
    localeId='es_ES',
    sessionId='test123',
    text='Quiero cancelar mi reserva'
)

response

# Information validation

You may also request information before assigning it...

In [34]:
# Slot: fecha
slot_fecha = lexv2.create_slot(
    botId=bot_id,
    botVersion="DRAFT",
    localeId=locale_id,
    intentId=intent_id,
    slotName="fecha",
    slotTypeId="AMAZON.Date",
    valueElicitationSetting={
        "slotConstraint": "Required",
        "promptSpecification":{
            "messageGroups":[{"message":{"plainTextMessage":{"value":"¿Para qué fecha quieres la reserva?"}}}],
            "maxRetries":2
        }
    }
)
fecha_id = slot_fecha['slotId']

# Slot: hora
slot_hora = lexv2.create_slot(
    botId=bot_id,
    botVersion="DRAFT",
    localeId=locale_id,
    intentId=intent_id,
    slotName="hora",
    slotTypeId="AMAZON.Time",
    valueElicitationSetting={
        "slotConstraint": "Required",
        "promptSpecification":{
            "messageGroups":[{"message":{"plainTextMessage":{"value":"¿A qué hora?"}}}],
            "maxRetries":2
        }
    }
)
hora_id = slot_hora['slotId']


In [ ]:
lexv2.update_intent(
    botId=bot_id,
    botVersion="DRAFT",
    intentName=intent_name,
    localeId=locale_id,
    intentId=intent_id,
    slotPriorities=[
        {"priority":1, "slotId": fecha_id},
        {"priority":2, "slotId": hora_id}
    ]
)


In [ ]:
# actualizamos bot
lexv2.build_bot_locale(botId=bot_id, botVersion="DRAFT", localeId=locale_id)


## Test

In [ ]:
response = lex_runtime.recognize_text(
    botId=bot_id,
    botAliasId='TSTALIASID',  # funciona con cualquiera
    localeId='es_ES',
    sessionId='test12',
    text='Quiero reservar una mesa para 3 personas'
)

response

In [ ]:
response['sessionState']

# Bot de voz


In [ ]:
# Voice: use RecognizeUtterance API

with open("respuesta_voz.wav", "wb") as audio_file:
    response = lex_runtime.recognize_utterance(
        botId=bot_id,
        botAliasId='TSTALIASID',
        localeId='es_ES',
        sessionId='voz123',
        requestContentType='text/plain; charset=utf-8',
        responseContentType='audio/mpeg',
        inputStream=b"Hola, quiero cancelar una reserva"
    )
    audio_file.write(response['audioStream'].read())


In [ ]:
# delete

#lexv2.delete_bot(botId=bot_id, skipResourceInUseCheck=True)